# HW2a: Java Calculator
- 이름: ???
- 학번: ????????

https://calculatorhistory.net

### Define your `Calculator` class by overriding abstract methods of the `ICalculator` interface (15점)

`ICalculator` 인터페이스의 주석에 지시된 내용과 수업시간에 설명한 내용을 바탕으로 해당 인터페이스를 구현하는 `Calculator`를 작성하고 적절히 테스트하라. 테스트들에서 인터페이스의 모든 메소드의 동작을 활용해야 하며, 의도한 테스트 시나리오대로 동작함을 확인할 수 있도록 적절한 출력이 이루어져야 한다.

`appendEntry` 메소드의 주석에 "legal entry"라는 표현이 등장하는데, Java 버전에서는 이 내용은 사실상 거의 무시해도 좋다. 이것은 Kotlin 버전에서만 구현하는 것으로 한다.
보통 계산기 응용프로그램들은 올바른 식(valid expression)으로서 구성될 가망이 없는 버튼을 누르면 해당 입력을 거부하거나, 혹은 사용자에게 친절을 베풀어 자동으로 사용자가 입력하지 않은 내용까지 자동으로 추가하여 올바른 식을 구성할 수 있는 형태로만 입력이 진행된다.
하지만, 이번 Java 버전에서는 올바른 식으로 구성될 가망이 없는 `Entry`들도 `appendEntry` 메소드를 통해 입력이 항상 가능하도록 구현하라.
그러니까 `appendEntry` 메소드는 항상 `true`를 리턴하게 된다는 말이다. 단, null인 `Entry`는 어떤 경우에도 추가되지 `appendEntry`의 파라메터가 null이 아님을 확인하고 진행하도록 java-forax 교재에서처럼 `Objects.requireNonNull`을 활용해 처리하라.

실제 계산기 응용프로그램과 `ICalculator` 인터페이스는 숫자를 입력할 때 차이점이 있다.
실제 계산기 응용프로그램은 43이라는 수를 입력하기 위해 각각의 자리수인 4를 입력하고 3을 입력하는 과정을 거쳐야 하며, 연속되는 자리수 이외의 다른 입력이 들어오면 이전까지 연속적으로 입력되었던 자리수들을 합쳐서 하나의 수를 구성한다.
우리는 이를 단순화하여 `Entry.Value` 값으로 한꺼번에 43이라는 수를 `appendEntry`로 추가하도록 한다.

`Calculator`의 기능을 구현하기 위해서는 `ICalculator`의 메소드를 구현하는 것 외에도 적절한 인스턴스 변수들도 선언되어야 할 것이다.

인스턴스 변수의 정보의 내용을 노출하는 메소드들은 캡슐화를 고려하여 작성하라.

### Override `Calculator::toString` (10점)

테스트 출력을 위해서는 적절히 보기좋게 계산기의 상태를 출력할 필요가 있다. 따라서 주어진 ICalculator 인터페이스의 추상 메소드를 오버라이딩하는 것 외에도 출력을 위한 `toString`도 적절히 오버라이드하라.

예컨대, `1 + 1`을 계산한 다음 `2 * 3`을 계산하고 나서 지금은 미완성인 식 `( 43 +`를 입력중인 상황이라면
Calculator 객체가 **정확히** 다음과 같이 출력되어야 한다.
들여쓰기에 해당하는 왼쪽의 빈칸들은 space가 아닌 tab으로 처리하라.

```
Calculator[
	entries="( 43 +",
	currVal=null,
	lastVal=6,
	history=[
		2 * 3 = 6,
		1 + 1 = 2
	]
]
```

Calculator의 기능을 완전히 완성하지 않더라도 이것만으로 부분점수가 부여될 수 있다.
예를 들면 `new Calculator()`로 객체를 생성한 직후에 history나 entries등을 빈 리스트 등의 정상적인 기본값이 아닌
테스트를 위해 임의로 초기화시켜 놓고 `toString`이 작동한다는 것을 시연하는 테스트가 있으면 부분점수 획득이 가능하다.
당연히 `toString`에 위 내용을 문자열로 하드코딩하면 당연히 인정이 안되고, 인스턴스 변수에 있는 데이터로부터 처리하여 문자열을 만들 내야 부분점수로 인정될 요건이 된다.

자신이 작성한 프로그램이 잘 동작하는지 알아보기 위해서는 테스트를 실행했을 때 적절한 데이터가 출력되는지 알아보아야 하는 기능이 필수적이다.
그러니까 작성하기 쉬운 간단한 메소드들은 먼저 작성해 놓더라도 조금 복잡하다고 생각되는 것을 작성하여 모든 내용을 완성하기 전에 일단 `toString`부터 작성할 것을 권장한다.

----

In [1]:
sealed interface Entry { // entries for arithmetic expressions
    enum BinOp implements Entry { ADD, MULT } // +, *
    enum Delim implements Entry { LPAREN, RPAREN } // (, )
    record Value(int number) implements Entry { }

    /////////////////////////////////////////
    // 아래는 실습 이후에 추가된 메소드 
    /////////////////////////////////////////
    
    static String asString(Entry e) {
        Objects.requireNonNull(e);
        return switch ( e.toString() ) {
                case "ADD" -> "+";
                case "MULT" -> "*";
                case "LPAREN" -> "(";
                case "RPAREN" -> ")";
                default -> ""+((Value)e).number();
        };
    }
}

In [2]:
Entry.BinOp.ADD

ADD

In [3]:
Entry.Delim.LPAREN

LPAREN

In [4]:
new Entry.Value(43)

Value[number=43]

In [5]:
Entry.asString( Entry.BinOp.ADD )

+

In [6]:
Entry.asString( new Entry.Value(43) )

43

In [7]:
import java.util.stream.*;

var v1 = new Entry.Value(43);
var v2 = new Entry.Value(56);
var es = List.of(v1, Entry.BinOp.MULT, Entry.Delim.LPAREN, v2, Entry.BinOp.ADD, v1, Entry.Delim.RPAREN );

System.out.println(es);

es.stream().map(Entry::asString).collect(Collectors.joining(" "))

[Value[number=43], MULT, LPAREN, Value[number=56], ADD, Value[number=43], RPAREN]


43 * ( 56 + 43 )

In [8]:
record Pair<A,B>(A first, B second) { } // because Java standard library does not provide pair data type

interface ICalculator {
    // Adds an entriy (either BinOp, Delim, or Vaule) at the end.
    // Appends legal entry input and return true (returns false for illegal input without adding it).
    // The parameter `e` is required to be non-null.
    // Further details are discussed in the homework assignment instruction above.
    boolean appendEntry(Entry e);
    
    // Removes the last added entry; The ◄(x or DEL) button
    // Removes the last input entry and returns true (returns false if not possible)
    boolean removeEntry();
    
    Entry.Value currValue(); // Currently calculated value from the current entries (defaults to null).
    Entry.Value lastValue(); // Last calculated value (defaults to 0).
    
    List<Entry> entries();  // Get current entries.
    List< Pair<List<Entry>, Entry.Value> > history();  // History of calculations.
    
    void clearEntries(); // Clear the current entries; AC button.
    void clearHistory(); // Clear history.
    
    /////////////////////////////////////////
    // 아래는 실습 이후에 추가된 메소드 
    /////////////////////////////////////////
    
    // The = button.
    // Adds the current entries and its calculated current value as a pair into history,
    // when the current entries form a valid expression to calculate the current vaule,
    // and clears the current entries and resets the current value.
    // Returns the calculated result value for adding valid form of entries and its result value.
    // Returns null, without adding to the history, when the current entries do not form a valid expression.
    Entry.Value calculate();
}

----
참고자료: https://madplay.github.io/post/call-javascript-function-from-java-using-graalvm

In [9]:
%maven org.graalvm.js:js:19.2.0.1
%maven org.graalvm.js:js-scriptengine:19.2.0.1

import org.graalvm.polyglot.*

In [10]:
try (Context context = Context.create("js")) {

    var value = context.eval("js", "3 + 4 * 10");
    System.out.println( value );
    System.out.println( value.toString().equals("43") );
    System.out.println( value.getClass() ); // 7

} catch (Exception e) {
    System.err.println(e);
}

43
true
class org.graalvm.polyglot.Value


In [11]:
Context context = Context.create("js"); // 그냥 항상 성공한다고 가정하면 이렇게 써도 ...

var value = context.eval("js", "3 + 4 * 10");
System.out.println( value );
System.out.println( value.toString().equals("43") );
System.out.println( value.getClass() );

int vi = value.asInt();
vi == 43

43
true
class org.graalvm.polyglot.Value


true

In [12]:
Context context = Context.create("js");

try {
    Value value = context.eval("js", "3 + 4 * "); // 사실 여기서 문제가 발생할 수도
    System.out.println( value );
    System.out.println( value.toString().equals("43") );
    System.out.println( value.getClass() );
} catch (Exception e) {
    System.err.println(e);
    System.out.println("이걸 적당히 잘 처리하면 유용하게 활용 가능");
}

org.graalvm.polyglot.PolyglotException: SyntaxError: Unnamed:1:8 Expected an operand but found eof
3 + 4 * 
        ^



이걸 적당히 잘 처리하면 유용하게 활용 가능


----

In [13]:
// ... class Calculator implements ICalculator ...